In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
import io

# --- 1. Definições da Arquitetura do Modelo ---
print("Definindo a arquitetura do modelo (ResBlock e ResNetSimulator)...")

class ResBlock(nn.Module):
    """
    Define um bloco ResNet básico
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.main_path = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut_path = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.shortcut_path = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.main_path(x) + self.shortcut_path(x)
        out = F.relu(out)
        return out

# N_OUTPUTS é quantos valores você quer prever.
N_OUTPUTS = 4 # (Fase, Transmitância)

class ResNetSimulator(nn.Module):
    """
    Implementação do Simulator baseado em ResNet.
    """
    def __init__(self, in_channels=1, n_outputs=N_OUTPUTS):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = ResBlock(64, 64, stride=1)
        self.layer2 = ResBlock(64, 128, stride=2)
        self.layer3 = ResBlock(128, 256, stride=2)
        self.layer4 = ResBlock(256, 256, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))

        self.head = nn.Sequential(
            nn.Linear(256 * 2 * 2, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, n_outputs)
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.head(out)
        return out

# --- 2. Definição da Função de Pré-processamento ---
print("Definindo a função de pré-processamento 'draw_meta_atom_ellipse'...")
IMG_SIZE = 64 # 64x64 pixels
MAX_DIM_NM = 520 # O tamanho físico total da célula unitária

def draw_meta_atom_ellipse(L_x_nm, L_y_nm):
    """
    Desenha um meta-átomo de ELIPSE como um array numpy binário.
    """
    scale_factor = IMG_SIZE / MAX_DIM_NM

    # O semi-eixo é metade do comprimento total
    a_px = (L_x_nm / 2.0) * scale_factor
    b_px = (L_y_nm / 2.0) * scale_factor

    if a_px == 0 or b_px == 0:
        return np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)

    x = np.arange(0, IMG_SIZE)
    y = np.arange(0, IMG_SIZE)
    xx, yy = np.meshgrid(x, y)

    center = (IMG_SIZE - 1) / 2.0

    term_x = ((xx - center) / a_px) ** 2
    term_y = ((yy - center) / b_px) ** 2

    mask = (term_x + term_y) <= 1

    img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    img[mask] = 1.0 # Define 1.0 (metal) em todos os pixels dentro da elipse

    return img

# --- 3. Carregar Modelo e Dados ---
print("Carregando o modelo e os pesos...")
try:
    # Nomes dos arquivos
    model_file = "/content/simulator_Definitivo.pth"
    data_file = "/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt"

    # Instanciar o modelo
    model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS)

    # Carregar os pesos (state_dict) - use map_location se não tiver GPU
    model_data = torch.load(model_file, map_location=torch.device('cpu'))

    if isinstance(model_data, nn.Module):
        model = model_data
        print("Arquivo .pth continha o objeto do modelo inteiro.")
    elif isinstance(model_data, dict):
        model.load_state_dict(model_data)
        print("Pesos do state_dict carregados com sucesso no modelo.")

    # Definir para modo de avaliação
    model.eval()

    # Carregar dados para validação
    df = pd.read_csv(data_file, sep='\t')
    print(f"Dados de validação '{data_file}' carregados.")

    # --- 4. Verificação de Precisão (ValidAção) ---
    print("\n--- Verificação de Precisão (primeiras 5 amostras) ---")

    df_sample = df.head()
    actuals = torch.from_numpy(df_sample[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32))

    image_list = []
    for _, row in df_sample.iterrows():
        img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
        image_list.append(img)

    image_batch = torch.from_numpy(np.array(image_list)).unsqueeze(1)

    with torch.no_grad():
        predictions = model(image_batch)

    print("\nPrevisões (Fase, Transmitancia):")
    print(predictions)
    print("\nValores Reais (Fase, Transmitancia):")
    print(actuals)

    mae = torch.mean(torch.abs(predictions - actuals), dim=0)
    print(f"\nErro Absoluto Médio (Amostra): Fase={mae[0].item():.6f}, Transmitancia={mae[1].item():.2f}")


    # --- 5. Simulação para Novos Valores ---
    print("\n--- Simulação para L_x = 210, L_y = 100 ---")

    L_x_new_nm = 210.0
    L_y_new_nm = 100.0

    # 1. Gerar a imagem para a nova entrada
    img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)

    # 2. Converter para o formato de tensor de batch (1, 1, 64, 64)
    input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

    # 3. Fazer a predição
    with torch.no_grad():
        pred_new = model(input_tensor)

    # 4. Extrair os valores
    predicted_fase_x = pred_new[1][0].item()
    predicted_fase_y = pred_new[1][1].item()
    predicted_transmitancia_x = pred_new[0][0].item()
    predicted_transmitancia_y = pred_new[0][1].item()

    print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
    print("==================================================")
    print("               Resultados da Simulação            ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")
    print(f" Transmitancia_x Prevista: {predicted_transmitancia_x:.4f}")
    print(f" Transmitancia_y Prevista: {predicted_transmitancia_y:.4f}")
    print(f" Fase_x Prevista: {predicted_fase_x:.6f}")
    print(f" Fase_y Prevista: {predicted_fase_y:.6f}")
    print("==================================================")

except Exception as e:
    print(f"\nOcorreu um erro durante a execução: {e}")
    import traceback
    print(traceback.format_exc())

Definindo a arquitetura do modelo (ResBlock e ResNetSimulator)...
Definindo a função de pré-processamento 'draw_meta_atom_ellipse'...
Carregando o modelo e os pesos...
Pesos do state_dict carregados com sucesso no modelo.
Dados de validação '/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt' carregados.

--- Verificação de Precisão (primeiras 5 amostras) ---

Previsões (Fase, Transmitancia):
tensor([[63.8094, 56.4566,  0.3013, -4.7718],
        [68.3098, 66.3456,  0.4405, -2.9015],
        [58.7497, 83.0725,  0.2732, -5.0449],
        [80.6861, 89.1231,  0.4615, -2.8628],
        [83.0166, 66.8216,  0.4608, -2.9043]])

Valores Reais (Fase, Transmitancia):
tensor([[104.5985,  85.8888,   0.5109,  -3.0616],
        [125.8549, 116.8805,   0.5173,   3.1301],
        [ 72.4869, 196.5058,   0.3603,  -2.7526],
        [159.5825, 174.7678,   0.5183,   2.9004],
        [167.6257, 118.5122,   0.5851,   2.9278]])

Erro Absoluto Médio (Amostra): Fase=55.115395, Transmitancia=66.15

--- Simulaçã

In [ ]:
# --- 5. Simulação para Novos Valores ---
print("\n--- Simulação para L_x = 210, L_y = 100 ---")

L_x_new_nm = 210.0
L_y_new_nm = 200.0

# 1. Gerar a imagem para a nova entrada
img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)

# 2. Converter para o formato de tensor de batch (1, 1, 64, 64)
input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

# 3. Fazer a predição
with torch.no_grad():
    pred_new = model(input_tensor)

# 4. Extrair os valores
predicted_fase = pred_new[0][0].item()
predicted_transmitancia = pred_new[0][1].item()

print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
print("==================================================")
print("               Resultados da Simulação            ")
print("==================================================")
print(f" L_x: {L_x_new_nm} nm")
print(f" L_y: {L_y_new_nm} nm")
print(f" Fase Prevista: {predicted_fase:.6f}")
print(f" Transmitancia Prevista: {predicted_transmitancia:.4f}")
print("==================================================")


--- Simulação para L_x = 210, L_y = 100 ---

Tensor de Saída Bruta: [[2.0806236e+18 3.7065625e+18]]
               Resultados da Simulação            
 L_x: 210.0 nm
 L_y: 200.0 nm
 Fase Prevista: 2080623582865522688.000000
 Transmitancia Prevista: 3706562548884045824.0000


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
import io

# --- 1. Definições da Arquitetura do Modelo ---
print("Definindo a arquitetura do modelo (ResBlock e ResNetSimulator)...")

class ResBlock(nn.Module):
    """
    Define um bloco ResNet básico
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.main_path = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut_path = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.shortcut_path = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.main_path(x) + self.shortcut_path(x)
        out = F.relu(out)
        return out

# N_OUTPUTS é quantos valores você quer prever.
N_OUTPUTS = 4 # 4 saídas: (T_x, T_y, F_x, F_y)

class ResNetSimulator(nn.Module):
    """
    Implementação do Simulator baseado em ResNet.
    """
    def __init__(self, in_channels=1, n_outputs=N_OUTPUTS):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = ResBlock(64, 64, stride=1)
        self.layer2 = ResBlock(64, 128, stride=2)
        self.layer3 = ResBlock(128, 256, stride=2)
        self.layer4 = ResBlock(256, 256, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))

        self.head = nn.Sequential(
            nn.Linear(256 * 2 * 2, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, n_outputs)
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.head(out)
        return out

# --- 2. Definição da Função de Pré-processamento ---
print("Definindo a função de pré-processamento 'draw_meta_atom_ellipse'...")
IMG_SIZE = 64 # 64x64 pixels
MAX_DIM_NM = 520 # O tamanho físico total da célula unitária

def draw_meta_atom_ellipse(L_x_nm, L_y_nm):
    """
    Desenha um meta-átomo de ELIPSE como um array numpy binário.
    """
    scale_factor = IMG_SIZE / MAX_DIM_NM

    # O semi-eixo é metade do comprimento total
    a_px = (L_x_nm / 2.0) * scale_factor
    b_px = (L_y_nm / 2.0) * scale_factor

    if a_px == 0 or b_px == 0:
        return np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)

    x = np.arange(0, IMG_SIZE)
    y = np.arange(0, IMG_SIZE)
    xx, yy = np.meshgrid(x, y)

    center = (IMG_SIZE - 1) / 2.0

    term_x = ((xx - center) / a_px) ** 2
    term_y = ((yy - center) / b_px) ** 2

    mask = (term_x + term_y) <= 1

    img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    img[mask] = 1.0

    return img

# --- 3. Carregar Modelo e Dados ---
print("Carregando o modelo e os pesos...")
try:
    # Nomes dos arquivos
    model_file = "/content/simulator_Definitivo.pth"
    data_file = "/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt"

    # Instanciar o modelo
    model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS)

    # Carregar os pesos (state_dict)
    model_data = torch.load(model_file, map_location=torch.device('cpu'))

    if isinstance(model_data, nn.Module):
        model = model_data
        print("Arquivo .pth continha o objeto do modelo inteiro.")
    elif isinstance(model_data, dict):
        model.load_state_dict(model_data)
        print("Pesos do state_dict carregados com sucesso no modelo.")

    model.eval()

    # Carregar dados para validação
    df = pd.read_csv(data_file, sep='\t')
    print(f"Dados de validação '{data_file}' carregados.")

    # --- 4. Verificação de Precisão (ValidAção) ---
    print("\n--- Verificação de Precisão (primeiras 5 amostras) ---")

    df_sample = df.head()
    # Ordem dos alvos: T_x, T_y, F_x, F_y
    actuals = torch.from_numpy(df_sample[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32))

    image_list = []
    for _, row in df_sample.iterrows():
        img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
        image_list.append(img)

    image_batch = torch.from_numpy(np.array(image_list)).unsqueeze(1)

    with torch.no_grad():
        predictions = model(image_batch)

    print("\nPrevisões (T_x, T_y, F_x, F_y):")
    print(predictions)
    print("\nValores Reais (T_x, T_y, F_x, F_y):")
    print(actuals)

    mae = torch.mean(torch.abs(predictions - actuals), dim=0)

    # --- CORREÇÃO AQUI ---
    # Imprimir todos os 4 erros médios
    print(f"\nErro Absoluto Médio (Amostra):")
    print(f"  T_x: {mae[0].item():.4f}, T_y: {mae[1].item():.4f}, F_x: {mae[2].item():.6f}, F_y: {mae[3].item():.6f}")
    # --- FIM DA CORREÇÃO ---


    # --- 5. Simulação para Novos Valores ---
    print("\n--- Simulação para L_x = 210, L_y = 100 ---")

    L_x_new_nm = 210.0
    L_y_new_nm = 100.0

    img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)
    input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        pred_new = model(input_tensor) # Saída tem shape (1, 4)

    # --- CORREÇÃO AQUI ---
    # Ler o tensor (1, 4) usando o índice [0] para o batch
    # e o segundo índice para a saída
    predicted_transmitancia_x = pred_new[0, 0].item()
    predicted_transmitancia_y = pred_new[0, 1].item()
    predicted_fase_x = pred_new[0, 2].item()
    predicted_fase_y = pred_new[0, 3].item()
    # --- FIM DA CORREÇÃO ---

    print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
    print("==================================================")
    print("           Resultados da Simulação                ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")
    print(f" Transmitancia_x Prevista: {predicted_transmitancia_x:.4f}")
    print(f" Transmitancia_y Prevista: {predicted_transmitancia_y:.4f}")
    print(f" Fase_x Prevista: {predicted_fase_x:.6f}")
    print(f" Fase_y Prevista: {predicted_fase_y:.6f}")

    print("==================================================")
    print("           Valores Esperados (Artificiais)        ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")

    # (Cálculos dos valores esperados - sem alteração)
    Transmitancia_x = np.sqrt(L_x_new_nm**2 + 123)
    Transmitancia_y = np.sqrt(L_y_new_nm**2 + 321)
    Fase_x = np.arctan2(L_x_new_nm + 123, L_y_new_nm + 321)
    Fase_y = np.arctan2(L_x_new_nm - 123, L_y_new_nm - 321)

    print(f" Transmitancia Esperada_x: {Transmitancia_x:.4f}")
    print(f" Transmitancia Esperada_y: {Transmitancia_y:.4f}")
    print(f" Fase Esperada_x: {Fase_x:.6f}")
    print(f" Fase Esperada_y: {Fase_y:.6f}")
    print("==================================================")

except Exception as e:
    print(f"\nOcorreu um erro durante a execução: {e}")
    import traceback
    print(traceback.format_exc())

Definindo a arquitetura do modelo (ResBlock e ResNetSimulator)...
Definindo a função de pré-processamento 'draw_meta_atom_ellipse'...
Carregando o modelo e os pesos...
Pesos do state_dict carregados com sucesso no modelo.
Dados de validação '/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt' carregados.

--- Verificação de Precisão (primeiras 5 amostras) ---

Previsões (T_x, T_y, F_x, F_y):
tensor([[63.8094, 56.4566,  0.3013, -4.7718],
        [68.3098, 66.3456,  0.4405, -2.9015],
        [58.7497, 83.0725,  0.2732, -5.0449],
        [80.6861, 89.1231,  0.4615, -2.8628],
        [83.0166, 66.8216,  0.4608, -2.9043]])

Valores Reais (T_x, T_y, F_x, F_y):
tensor([[104.5985,  85.8888,   0.5109,  -3.0616],
        [125.8549, 116.8805,   0.5173,   3.1301],
        [ 72.4869, 196.5058,   0.3603,  -2.7526],
        [159.5825, 174.7678,   0.5183,   2.9004],
        [167.6257, 118.5122,   0.5851,   2.9278]])

Erro Absoluto Médio (Amostra):
  T_x: 55.1154, T_y: 66.1471, F_x: 0.110912, F_y: 4

In [ ]:
# --- 3. Carregar Modelo e Dados ---
print("Carregando o modelo e os pesos...")
try:
    # Nomes dos arquivos
    model_file = "/content/simulator_Definitivo.pth"
    data_file = "/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt"

    # Instanciar o modelo
    model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS)

    # Carregar os pesos (state_dict)
    model_data = torch.load(model_file, map_location=torch.device('cpu'))

    if isinstance(model_data, nn.Module):
        model = model_data
        print("Arquivo .pth continha o objeto do modelo inteiro.")
    elif isinstance(model_data, dict):
        model.load_state_dict(model_data)
        print("Pesos do state_dict carregados com sucesso no modelo.")

    model.eval()

    # Carregar dados para validação
    df = pd.read_csv(data_file, sep='\t')
    print(f"Dados de validação '{data_file}' carregados.")

    # --- 4. Verificação de Precisão (ValidAção) ---
    print("\n--- Verificação de Precisão (primeiras 5 amostras) ---")

    df_sample = df.head()
    # Ordem dos alvos: T_x, T_y, F_x, F_y
    actuals = torch.from_numpy(df_sample[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32))

    image_list = []
    for _, row in df_sample.iterrows():
        img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
        image_list.append(img)

    image_batch = torch.from_numpy(np.array(image_list)).unsqueeze(1)

    with torch.no_grad():
        predictions = model(image_batch)

    print("\nPrevisões (T_x, T_y, F_x, F_y):")
    print(predictions)
    print("\nValores Reais (T_x, T_y, F_x, F_y):")
    print(actuals)

    mae = torch.mean(torch.abs(predictions - actuals), dim=0)

    # --- CORREÇÃO AQUI ---
    # Imprimir todos os 4 erros médios
    print(f"\nErro Absoluto Médio (Amostra):")
    print(f"  T_x: {mae[0].item():.4f}, T_y: {mae[1].item():.4f}, F_x: {mae[2].item():.6f}, F_y: {mae[3].item():.6f}")
    # --- FIM DA CORREÇÃO ---


    # --- 5. Simulação para Novos Valores ---
    print("\n--- Simulação para L_x = 210, L_y = 100 ---")

    L_x_new_nm = 150.0
    L_y_new_nm = 200.0

    img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)
    input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        pred_new = model(input_tensor) # Saída tem shape (1, 4)

    # --- CORREÇÃO AQUI ---
    # Ler o tensor (1, 4) usando o índice [0] para o batch
    # e o segundo índice para a saída
    predicted_transmitancia_x = pred_new[0, 0].item()
    predicted_transmitancia_y = pred_new[0, 1].item()
    predicted_fase_x = pred_new[0, 2].item()
    predicted_fase_y = pred_new[0, 3].item()
    # --- FIM DA CORREÇÃO ---

    print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
    print("==================================================")
    print("           Resultados da Simulação                ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")
    print(f" Transmitancia_x Prevista: {predicted_transmitancia_x:.4f}")
    print(f" Transmitancia_y Prevista: {predicted_transmitancia_y:.4f}")
    print(f" Fase_x Prevista: {predicted_fase_x:.6f}")
    print(f" Fase_y Prevista: {predicted_fase_y:.6f}")

    print("==================================================")
    print("           Valores Esperados (Artificiais)        ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")

    # (Cálculos dos valores esperados - sem alteração)
    Transmitancia_x = np.sqrt(L_x_new_nm**2 + 123)
    Transmitancia_y = np.sqrt(L_y_new_nm**2 + 321)
    Fase_x = np.arctan2(L_x_new_nm + 123, L_y_new_nm + 321)
    Fase_y = np.arctan2(L_x_new_nm - 123, L_y_new_nm - 321)

    print(f" Transmitancia Esperada_x: {Transmitancia_x:.4f}")
    print(f" Transmitancia Esperada_y: {Transmitancia_y:.4f}")
    print(f" Fase Esperada_x: {Fase_x:.6f}")
    print(f" Fase Esperada_y: {Fase_y:.6f}")
    print("==================================================")

except Exception as e:
    print(f"\nOcorreu um erro durante a execução: {e}")
    import traceback
    print(traceback.format_exc())

Carregando o modelo e os pesos...
Pesos do state_dict carregados com sucesso no modelo.
Dados de validação '/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt' carregados.

--- Verificação de Precisão (primeiras 5 amostras) ---

Previsões (T_x, T_y, F_x, F_y):
tensor([[63.8094, 56.4566,  0.3013, -4.7718],
        [68.3098, 66.3456,  0.4405, -2.9015],
        [58.7497, 83.0725,  0.2732, -5.0449],
        [80.6861, 89.1231,  0.4615, -2.8628],
        [83.0166, 66.8216,  0.4608, -2.9043]])

Valores Reais (T_x, T_y, F_x, F_y):
tensor([[104.5985,  85.8888,   0.5109,  -3.0616],
        [125.8549, 116.8805,   0.5173,   3.1301],
        [ 72.4869, 196.5058,   0.3603,  -2.7526],
        [159.5825, 174.7678,   0.5183,   2.9004],
        [167.6257, 118.5122,   0.5851,   2.9278]])

Erro Absoluto Médio (Amostra):
  T_x: 55.1154, T_y: 66.1471, F_x: 0.110912, F_y: 4.325895

--- Simulação para L_x = 210, L_y = 100 ---

Tensor de Saída Bruta: [[ 75.578445   100.57314      0.44448787  -2.8231578 ]]
 

In [ ]:
# --- 3. Carregar Modelo e Dados ---
print("Carregando o modelo e os pesos...")
try:
    # Nomes dos arquivos
    model_file = "/content/simulator_Definitivo.pth"
    data_file = "/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt"

    # Instanciar o modelo
    model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS)

    # Carregar os pesos (state_dict)
    model_data = torch.load(model_file, map_location=torch.device('cpu'))

    if isinstance(model_data, nn.Module):
        model = model_data
        print("Arquivo .pth continha o objeto do modelo inteiro.")
    elif isinstance(model_data, dict):
        model.load_state_dict(model_data)
        print("Pesos do state_dict carregados com sucesso no modelo.")

    model.eval()

    # Carregar dados para validação
    df = pd.read_csv(data_file, sep='\t')
    print(f"Dados de validação '{data_file}' carregados.")

    # --- 4. Verificação de Precisão (ValidAção) ---
    print("\n--- Verificação de Precisão (primeiras 5 amostras) ---")

    df_sample = df.head()
    # Ordem dos alvos: T_x, T_y, F_x, F_y
    actuals = torch.from_numpy(df_sample[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32))

    image_list = []
    for _, row in df_sample.iterrows():
        img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
        image_list.append(img)

    image_batch = torch.from_numpy(np.array(image_list)).unsqueeze(1)

    with torch.no_grad():
        predictions = model(image_batch)

    print("\nPrevisões (T_x, T_y, F_x, F_y):")
    print(predictions)
    print("\nValores Reais (T_x, T_y, F_x, F_y):")
    print(actuals)

    mae = torch.mean(torch.abs(predictions - actuals), dim=0)

    # --- CORREÇÃO AQUI ---
    # Imprimir todos os 4 erros médios
    print(f"\nErro Absoluto Médio (Amostra):")
    print(f"  T_x: {mae[0].item():.4f}, T_y: {mae[1].item():.4f}, F_x: {mae[2].item():.6f}, F_y: {mae[3].item():.6f}")
    # --- FIM DA CORREÇÃO ---


    # --- 5. Simulação para Novos Valores ---
    print("\n--- Simulação para L_x = 210, L_y = 100 ---")

    L_x_new_nm = 70
    L_y_new_nm = 70

    img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)
    input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        pred_new = model(input_tensor) # Saída tem shape (1, 4)

    # --- CORREÇÃO AQUI ---
    # Ler o tensor (1, 4) usando o índice [0] para o batch
    # e o segundo índice para a saída
    predicted_transmitancia_x = pred_new[0, 0].item()
    predicted_transmitancia_y = pred_new[0, 1].item()
    predicted_fase_x = pred_new[0, 2].item()
    predicted_fase_y = pred_new[0, 3].item()
    # --- FIM DA CORREÇÃO ---

    print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
    print("==================================================")
    print("           Resultados da Simulação                ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")
    print(f" Transmitancia_x Prevista: {predicted_transmitancia_x:.4f}")
    print(f" Transmitancia_y Prevista: {predicted_transmitancia_y:.4f}")
    print(f" Fase_x Prevista: {predicted_fase_x:.6f}")
    print(f" Fase_y Prevista: {predicted_fase_y:.6f}")

    print("==================================================")
    print("           Valores Esperados (Artificiais)        ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")

    # (Cálculos dos valores esperados - sem alteração)
    Transmitancia_x = np.sqrt(L_x_new_nm**2 + 123)
    Transmitancia_y = np.sqrt(L_y_new_nm**2 + 321)
    Fase_x = np.arctan2(L_x_new_nm + 123, L_y_new_nm + 321)
    Fase_y = np.arctan2(L_x_new_nm - 123, L_y_new_nm - 321)

    print(f" Transmitancia Esperada_x: {Transmitancia_x:.4f}")
    print(f" Transmitancia Esperada_y: {Transmitancia_y:.4f}")
    print(f" Fase Esperada_x: {Fase_x:.6f}")
    print(f" Fase Esperada_y: {Fase_y:.6f}")
    print("==================================================")

except Exception as e:
    print(f"\nOcorreu um erro durante a execução: {e}")
    import traceback
    print(traceback.format_exc())

Carregando o modelo e os pesos...
Pesos do state_dict carregados com sucesso no modelo.
Dados de validação '/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt' carregados.

--- Verificação de Precisão (primeiras 5 amostras) ---

Previsões (T_x, T_y, F_x, F_y):
tensor([[63.8094, 56.4566,  0.3013, -4.7718],
        [68.3098, 66.3456,  0.4405, -2.9015],
        [58.7497, 83.0725,  0.2732, -5.0449],
        [80.6861, 89.1231,  0.4615, -2.8628],
        [83.0166, 66.8216,  0.4608, -2.9043]])

Valores Reais (T_x, T_y, F_x, F_y):
tensor([[104.5985,  85.8888,   0.5109,  -3.0616],
        [125.8549, 116.8805,   0.5173,   3.1301],
        [ 72.4869, 196.5058,   0.3603,  -2.7526],
        [159.5825, 174.7678,   0.5183,   2.9004],
        [167.6257, 118.5122,   0.5851,   2.9278]])

Erro Absoluto Médio (Amostra):
  T_x: 55.1154, T_y: 66.1471, F_x: 0.110912, F_y: 4.325895

--- Simulação para L_x = 210, L_y = 100 ---

Tensor de Saída Bruta: [[56.090622   54.97387     0.26609856 -4.8791656 ]]
     

In [ ]:
# --- 3. Carregar Modelo e Dados ---
print("Carregando o modelo e os pesos...")
try:
    # Nomes dos arquivos
    model_file = "/content/simulator_Definitivo.pth"
    data_file = "/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt"

    # Instanciar o modelo
    model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS)

    # Carregar os pesos (state_dict)
    model_data = torch.load(model_file, map_location=torch.device('cpu'))

    if isinstance(model_data, nn.Module):
        model = model_data
        print("Arquivo .pth continha o objeto do modelo inteiro.")
    elif isinstance(model_data, dict):
        model.load_state_dict(model_data)
        print("Pesos do state_dict carregados com sucesso no modelo.")

    model.eval()

    # Carregar dados para validação
    df = pd.read_csv(data_file, sep='\t')
    print(f"Dados de validação '{data_file}' carregados.")

    # --- 4. Verificação de Precisão (ValidAção) ---
    print("\n--- Verificação de Precisão (primeiras 5 amostras) ---")

    df_sample = df.head()
    # Ordem dos alvos: T_x, T_y, F_x, F_y
    actuals = torch.from_numpy(df_sample[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32))

    image_list = []
    for _, row in df_sample.iterrows():
        img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
        image_list.append(img)

    image_batch = torch.from_numpy(np.array(image_list)).unsqueeze(1)

    with torch.no_grad():
        predictions = model(image_batch)

    print("\nPrevisões (T_x, T_y, F_x, F_y):")
    print(predictions)
    print("\nValores Reais (T_x, T_y, F_x, F_y):")
    print(actuals)

    mae = torch.mean(torch.abs(predictions - actuals), dim=0)

    # --- CORREÇÃO AQUI ---
    # Imprimir todos os 4 erros médios
    print(f"\nErro Absoluto Médio (Amostra):")
    print(f"  T_x: {mae[0].item():.4f}, T_y: {mae[1].item():.4f}, F_x: {mae[2].item():.6f}, F_y: {mae[3].item():.6f}")
    # --- FIM DA CORREÇÃO ---


    # --- 5. Simulação para Novos Valores ---
    print("\n--- Simulação para L_x = 210, L_y = 100 ---")

    L_x_new_nm = 200.0
    L_y_new_nm = 200.0

    img_new = draw_meta_atom_ellipse(L_x_new_nm, L_y_new_nm)
    input_tensor = torch.from_numpy(img_new).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        pred_new = model(input_tensor) # Saída tem shape (1, 4)

    # --- CORREÇÃO AQUI ---
    # Ler o tensor (1, 4) usando o índice [0] para o batch
    # e o segundo índice para a saída
    predicted_transmitancia_x = pred_new[0, 0].item()
    predicted_transmitancia_y = pred_new[0, 1].item()
    predicted_fase_x = pred_new[0, 2].item()
    predicted_fase_y = pred_new[0, 3].item()
    # --- FIM DA CORREÇÃO ---

    print(f"\nTensor de Saída Bruta: {pred_new.numpy()}")
    print("==================================================")
    print("           Resultados da Simulação                ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")
    print(f" Transmitancia_x Prevista: {predicted_transmitancia_x:.4f}")
    print(f" Transmitancia_y Prevista: {predicted_transmitancia_y:.4f}")
    print(f" Fase_x Prevista: {predicted_fase_x:.6f}")
    print(f" Fase_y Prevista: {predicted_fase_y:.6f}")

    print("==================================================")
    print("           Valores Esperados (Artificiais)        ")
    print("==================================================")
    print(f" L_x: {L_x_new_nm} nm")
    print(f" L_y: {L_y_new_nm} nm")

    # (Cálculos dos valores esperados - sem alteração)
    Transmitancia_x = np.sqrt(L_x_new_nm**2 + 123)
    Transmitancia_y = np.sqrt(L_y_new_nm**2 + 321)
    Fase_x = np.arctan2(L_x_new_nm + 123, L_y_new_nm + 321)
    Fase_y = np.arctan2(L_x_new_nm - 123, L_y_new_nm - 321)

    print(f" Transmitancia Esperada_x: {Transmitancia_x:.4f}")
    print(f" Transmitancia Esperada_y: {Transmitancia_y:.4f}")
    print(f" Fase Esperada_x: {Fase_x:.6f}")
    print(f" Fase Esperada_y: {Fase_y:.6f}")
    print("==================================================")

except Exception as e:
    print(f"\nOcorreu um erro durante a execução: {e}")
    import traceback
    print(traceback.format_exc())

Carregando o modelo e os pesos...
Pesos do state_dict carregados com sucesso no modelo.
Dados de validação '/content/df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt' carregados.

--- Verificação de Precisão (primeiras 5 amostras) ---

Previsões (T_x, T_y, F_x, F_y):
tensor([[63.8094, 56.4566,  0.3013, -4.7718],
        [68.3098, 66.3456,  0.4405, -2.9015],
        [58.7497, 83.0725,  0.2732, -5.0449],
        [80.6861, 89.1231,  0.4615, -2.8628],
        [83.0166, 66.8216,  0.4608, -2.9043]])

Valores Reais (T_x, T_y, F_x, F_y):
tensor([[104.5985,  85.8888,   0.5109,  -3.0616],
        [125.8549, 116.8805,   0.5173,   3.1301],
        [ 72.4869, 196.5058,   0.3603,  -2.7526],
        [159.5825, 174.7678,   0.5183,   2.9004],
        [167.6257, 118.5122,   0.5851,   2.9278]])

Erro Absoluto Médio (Amostra):
  T_x: 55.1154, T_y: 66.1471, F_x: 0.110912, F_y: 4.325895

--- Simulação para L_x = 210, L_y = 100 ---

Tensor de Saída Bruta: [[ 99.9897    100.10385     0.4623018  -3.073245 ]]
     